In [32]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import warnings

In [ ]:
pred_dir = Path('./videos/video_1/keypoints_json') # directory including keypoints.json files
gt_dir   = Path('./videos/video_1/keypoints_json') # directory including keypoints.json files
thresh   = 0.2   # Threshold ratio
person_idx = 0   # Person index (0-base)

pred_files = sorted(pred_dir.glob('*.json'))
gt_files   = sorted(gt_dir.glob('*.json'))

pair_cnt = min(len(pred_files), len(gt_files))
if len(pred_files) != len(gt_files):
    warnings.warn(f'Directory sizes differ (pred {len(pred_files)}, gt {len(gt_files)}). Evaluating {pair_cnt} pairs.')

results = []

In [34]:

for idx in range(pair_cnt):
    pred_path = pred_files[idx]
    gt_path   = gt_files[idx]
    try:
        # --- Load keypoints ---
        with open(pred_path, 'r') as f:
            pred_data = json.load(f)
        with open(gt_path, 'r') as f:
            gt_data = json.load(f)

        pred_people = pred_data.get('people', [])
        gt_people   = gt_data.get('people', [])

        if not pred_people or not gt_people:
            raise ValueError('No people in one of the files.')

        if person_idx >= len(pred_people) or person_idx >= len(gt_people):
            raise IndexError('person_index out of range.')

        pred_arr = np.asarray(pred_people[person_idx]['pose_keypoints_2d']).reshape(-1, 3)
        gt_arr   = np.asarray(gt_people[person_idx]['pose_keypoints_2d']).reshape(-1, 3)

        kp_pred = pred_arr[:, :2]
        kp_gt   = gt_arr[:, :2]
        conf_gt = gt_arr[:, 2]

        # --- Torso length (L-Shoulder 5 ↔ R-Hip 9) ---
        torso_len = np.linalg.norm(kp_gt[5] - kp_gt[9])
        if torso_len == 0:
            raise ValueError('Zero torso length in GT.')

        valid = conf_gt > 0.05
        dists = np.linalg.norm(kp_pred[valid] - kp_gt[valid], axis=1)
        correct = dists < thresh * torso_len
        pck_val = correct.mean()

        results.append({'index': int(idx),
                        'pred_file': str(pred_path.name),
                        'gt_file':   str(gt_path.name),
                        'PCK': float(pck_val)})
    except Exception as e:
        results.append({'index': int(idx),
                        'pred_file': str(pred_path.name),
                        'gt_file':   str(gt_path.name),
                        'PCK': np.nan,
                        'error': str(e)})


In [35]:
valid_pck = [r["PCK"] for r in results if isinstance(r["PCK"], float) and not np.isnan(r["PCK"])]
mean_pck_list = np.mean(valid_pck) if valid_pck else float("nan")
print(f"\nMean PCK@{thresh:.1f} = {mean_pck_list*100:.2f}%")


Mean PCK@0.2 = 100.00%


In [36]:
results

[{'index': 0,
  'pred_file': 'frame_000000_keypoints.json',
  'gt_file': 'frame_000000_keypoints.json',
  'PCK': 1.0},
 {'index': 1,
  'pred_file': 'frame_000001_keypoints.json',
  'gt_file': 'frame_000001_keypoints.json',
  'PCK': 1.0},
 {'index': 2,
  'pred_file': 'frame_000002_keypoints.json',
  'gt_file': 'frame_000002_keypoints.json',
  'PCK': 1.0},
 {'index': 3,
  'pred_file': 'frame_000003_keypoints.json',
  'gt_file': 'frame_000003_keypoints.json',
  'PCK': 1.0},
 {'index': 4,
  'pred_file': 'frame_000004_keypoints.json',
  'gt_file': 'frame_000004_keypoints.json',
  'PCK': 1.0},
 {'index': 5,
  'pred_file': 'frame_000005_keypoints.json',
  'gt_file': 'frame_000005_keypoints.json',
  'PCK': 1.0},
 {'index': 6,
  'pred_file': 'frame_000006_keypoints.json',
  'gt_file': 'frame_000006_keypoints.json',
  'PCK': 1.0},
 {'index': 7,
  'pred_file': 'frame_000007_keypoints.json',
  'gt_file': 'frame_000007_keypoints.json',
  'PCK': 1.0},
 {'index': 8,
  'pred_file': 'frame_000008_keypo